<a href="https://colab.research.google.com/github/Eldave93/Seizure-Detection-Tutorials/blob/master/Extra_01_Assemble_Feature_DataFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extra - Assemble Feature DataFrames

by [David Luke Elliott](https://www.lancaster.ac.uk/psychology/about-us/people/david-elliott)
/ [GitHub](https://github.com/Eldave93)

Instead of running this, you can just download the datasets from my google drive as detailed in the tutorial notebooks. 

During making these datasets you will save things onto the disk in colab, but it will be deleted when you (re-)connect to an instance. Therefore you may want to connect a google drive (as seen below) or use the left tab to save the data (locate the file in Files, then right click the file and download).

__Note__
- Due to time changing things, the data you get out of this will likely be different from the datasets used in the tutorials currently. I aim to updated them but I haven't had time to check they don't break the tutorials so I'll leave them as is for now.
- Also if you are planning on using some of this code for your research, go through it very carefully and make changes where needed. I threw this together from various bits of old code into one file, I've probably made a mistake somewhere (hopefully in the future I'll get time to come back and improve it and hunt down my scripts and find if/where i fixed things).
- Some of these are not the most efficient way of doing things so they typically download more data than you need or take longer than they could... just a warning if you intend to run them.

In [1]:
!pip install matplotlib pandas==0.23.0
!pip install numpy scipy seaborn mne
!pip install beautifulsoup4 requests wget
!pip install h5py tables kaggle
!pip install wfdb pyEDFlib PyWavelets

     |████████████████████████████████| 11.7MB 10.3MB/s 
ERROR: xarray 0.15.1 has requirement pandas>=0.25, but you'll have pandas 0.23.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.23.0 which is incompatible.
ERROR: mizani 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.23.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 0.23.0 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement pandas>=1.0.4, but you'll have pandas 0.23.0 which is incompatible.
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


     |████████████████████████████████| 6.9MB 9.9MB/s 
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=550b90ce1e5d87f16d48c3df8f6c01d29b5bfcd88b46465da10e43bf3b34f460
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 122kB 10.1MB/s 
     |████████████████████████████████| 1.2MB 22.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyEDFlib: filename=pyEDFlib-0.1.20-cp36-cp36m-linux_x86_64.whl size=928463 sha256=b99ee3e22b849aef403307c6c0002661239178b0d5f07bc8907056f0d53dfa27
  Stored in directory: /root/.cache/pip/wheels/0d/be/b5/574f8b99ad9ac2ecf78e0c5271040b5aff999febfe655b97a4
Successfully built pyEDFlib


Clone and import my helper function.

In [1]:
!git clone https://github.com/Eldave93/Seizure-Feature-Extraction

Cloning into 'Seizure-Feature-Extraction'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 10), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [1]:
import sys, os

sys.path.append(os.path.join(os.getcwd(), "Seizure-Feature-Extraction"))

In [2]:
#TODO: Tidy imports up
import glob            # for file locations
import pprint          # for pretty printing
import re
from getpass import getpass
import pyedflib
from collections import Counter
from pywt import swt
import math
from bs4 import BeautifulSoup
from Seizure_Feature_Extraction import Seizure_Features
import sys
import os
from bs4 import BeautifulSoup
import requests
import re
import wget
import zipfile
import pandas as pd     # dataframes
from tqdm import tqdm_notebook as tqdm
from getpass import getpass
import os
from scipy.io import loadmat
import mne
import numpy as np
import tables
import numpy as np
import tables
import os
from urllib.request import urlretrieve
import wfdb 
import re
import pandas as pd
import numpy as np
import pyedflib
from sklearn.preprocessing import StandardScaler
from scipy import signal
import sys
import warnings
from tqdm import tqdm_notebook as tqdm

# colours for printing outputs
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

pp = pprint.PrettyPrinter()

def file_list(folder_path, output=False):
    # create an empty list
    file_list = []
    # for file name in the folder path...
    for filename in glob.glob(folder_path):
        # ... append it to the list
        file_list.append(filename)
        
    # sort alphabetically
    file_list.sort()
    
    # Output
    if output:
        print(str(len(file_list)) + " files found")
        pp.pprint(file_list)
    
    return file_list

In [3]:
def window(a, w, o, copy = False):
  # if there is no window to be applied
  if w == None:
      view = np.expand_dims(a.T, axis=0)

  # otherwise...
  else:
      sh = (a.size - w + 1, w)
      st = a.strides * 2
      if o:
          view = np.lib.stride_tricks.as_strided(a, strides = st, shape = sh)[0::o]
      else:
          view = np.lib.stride_tricks.as_strided(a, strides = st, shape = sh)[0::w]
  if copy:
      return view.copy()
  else:
      return view
    
def window_x(data, window_size, overlap):
    
  for i, column in enumerate(data.columns):

    # window the data so each row is another epoch
    channel_windowed = window(data[column], w = window_size, o = overlap, copy = True)
    channel_windowed = np.reshape(channel_windowed, (*channel_windowed.shape, -1))

    if i ==0:
      windowed_data = channel_windowed
    else:
      windowed_data = np.concatenate((windowed_data, channel_windowed), axis=-1)
  
  return windowed_data

# Save Paths

Mount using the mount drive in the top left for some reason below wasnt keeping up...

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Epileptologie

We're creating this combine function to take all the data from the different feature extraction methods and combine them into one dataframe.

Now below is the code to take all those functions we have worked on so far and use them on all the data, and then combining the outputs into one big dataframe.

**NOTES**
- for me it takes 12 s
- we use JSON rather than pickle (which can compress it down more) to save because JSON does not introduce a security hole and allows the data to be used by programs in different languages.

In [ ]:
# Run the code and create/overwite old data?
EPIL_OVERWRITE = True

# Where to save the feature data
EPIL_SAVE_PATH = '/content/drive/My Drive/Data/Epil_features.json.gzip'

In [ ]:
def find_files(url):
    # get a soup of the directory url
    soup = BeautifulSoup(requests.get(url).text, features="html.parser")

    # make a list of all the links in the url
    hrefs_list = []
    for link in soup.find_all('a'):
        hrefs_list.append(link.get('href'))

    return hrefs_list
    
    
def download_file(download_file_url, file_dir, output=False):
    if output:
        # print it is downloading
        print('Downloading: '+ download_file_url)
    # download the file to the directory
    wget.download(download_file_url, file_dir)
    
    
# needs a directory to download it to
def download_epileptologie(DIR, output=False):
    
    # directory url
    front_url = 'http://epileptologie-bonn.de/cms/front_content.php?idcat=193&lang=3&changelang=3'
    dir_url = 'http://epileptologie-bonn.de/cms'

    hrefs_dir_list = find_files(front_url)
    
    # for each link in the directory
    for link in hrefs_dir_list:
        # download the files outside of participant folders we want
        if re.findall('zip', str(link)):
            # if the file doesnt already exist in the directory
            if not os.path.exists(os.path.join(DIR, link)):
                download_file(dir_url+'/'+str(link), DIR, output)
                zip_file_name = link.split('/')[-1]
                zip_ref = zipfile.ZipFile(os.path.join(DIR, zip_file_name), 'r')
                zip_ref.extractall(os.path.join(DIR, zip_file_name[0]))
                zip_ref.close()
                os.remove(os.path.join(DIR, zip_file_name))

def data_load(file_path, output=False):

    # read in the datafile
    data = pd.read_csv(file_path,                 # file in
                       header=None,               # no column names at top of file
                       dtype=float)               # read data as 'floating points' (e.g. 1.0)

    if output:
        print(color.BOLD+color.UNDERLINE+'\n'+re.findall('\w\d+',file_path)[0]+color.END)
        # Output detailed information on the data
        print(color.BOLD+'\nData Information'+color.END)
        data.info()

        # Output first 5 rows and columns
        print(color.BOLD+'\nDataframe Head'+color.END)
        display(data.head())
        
    return data


def data_index(feat_data, file_name, output = False):
    
    # get the file identifier from the file (e.g. F001)
    file_identifier = re.findall('\w\d+', file_name)[0]
    # add this identifier to a column
    feat_data['file_id'] = file_identifier
    
    # if the file identifier has an S in...
    if re.findall('S', file_identifier):
        # make a class column with 'seizure' in
        feat_data['class'] = 'seizure'
    # ...otherwise...
    else:
        # .. make a class column with 'Baseline' in
        feat_data['class'] = 'baseline'
        
    
    # if the file identifier has a Z or O in...
    if re.findall('Z|O', file_identifier):
        # make a location column with 'surface' in
        feat_data['location'] = 'surface'
    # if the file identifier has an N in...
    elif re.findall('N', file_identifier):
        # make a location column with 'intracranial hippocampus' in
        feat_data['location'] = 'intracranial hippocampus'
    # if the file identifier has an S or F in...
    elif re.findall('F|S', file_identifier):
        # make a location column with 'intracranial epileptogenic zone' in
        feat_data['location'] = 'intracranial epileptogenic zone'
        
    # name the index
    feat_data.columns.name = 'feature'
    
    # add the file_id and class to the index
    feat_data = feat_data.set_index(['file_id', 'class', 'location'])
    # reorder the index so class is first, then file_id, then feature
    feat_data = feat_data.reorder_levels(['class', 'location', 'file_id'], axis='index')
    
    if output:
        display(feat_data)
        
    return feat_data


if EPIL_OVERWRITE:
  # delete the old version
  if os.path.exists(EPIL_SAVE_PATH):
      os.remove(EPIL_SAVE_PATH)

  # --------
  # DOWNLOAD
  # --------
  DOWNLOAD_DIR = "Epileptologie Database"

  if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)
    download_epileptologie(DOWNLOAD_DIR, output=True)

  else:
    print("Already Downloaded")

  # get a list of the downloaded files
  EPIL_dir_file_list = file_list(os.path.join(DOWNLOAD_DIR, '*'), output=False)
  
  # -----------------
  # CREATE FEATURE DF
  # -----------------
  # make an empty dataframe
  feature_df = pd.DataFrame()
  # iterate across the list of folders
  for folder in tqdm(EPIL_dir_file_list):
      # get a list of files in each folder
      folder_files_list = file_list(os.path.join(folder, '*'))
      # iteratate across the files in each folder
      for file in folder_files_list:
          # load the file
          df = data_load(file)
          # setup the feature extraction function
          feat = Seizure_Features(sf = 173.61, 
                          window_size=None,
                          feature_list=['power', 'power_ratio', 'mean', 
                                        'mean_abs', 'std','ratio', 'LSWT'],
                          bandpasses = [[2,4],[4,8],[8,12], [12,30],[30,70]])
          # transform the data using the function
          part_x_feat = feat.transform(df.values, channel_names_list = ['CZ'])
          # put the numpy output back into a pandas df
          part_x_feat = pd.DataFrame(part_x_feat, columns = feat.feature_names)
          # re-index the data
          part_x_feat = data_index(part_x_feat, file)
          # if there is no data in the feature data so far...
          if feature_df.empty:
              # then make this the feature dataframe...
              feature_df = part_x_feat
          else:
              # ...otherwise combine the two dataframes together down the index
              feature_df = pd.concat([feature_df, part_x_feat], axis='index')

  # display the dataframe
  display(feature_df)

  # reset the index into columns (for easy saving)
  feature_df_save = feature_df.reset_index()

  # save the dataframe to disk for later use
  feature_df_save.to_json(EPIL_SAVE_PATH, 
                          orient='index', 
                          compression = 'gzip') 

Downloading: http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/Z.zip
Downloading: http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/O.zip
Downloading: http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/N.zip
Downloading: http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/F.zip
Downloading: http://epileptologie-bonn.de/cms/upload/workgroup/lehnertz/S.zip


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:143: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


feature                                              CZ|2_4Hz  ...  CZ|D1_LSWT
class    location                        file_id               ...            
baseline intracranial epileptogenic zone F001       77.429564  ...    3.665018
                                         F002     1060.585071  ...    8.020356
                                         F003      316.076591  ...    7.170449
                                         F004      213.966621  ...    6.950912
                                         F005      973.296659  ...    8.480473
...                                                       ...  ...         ...
         surface                         Z096       84.704705  ...    6.180514
                                         Z097       94.787882  ...    7.163136
                                         Z098       81.554287  ...    6.122005
                                         Z099       89.632144  ...    4.568267
                                         Z100      111.931878  ...    6.255625

[500 rows x 36 columns]

## UPenn and Mayo

**TODO**
- add a `save_progress` flag for the function. This makes a DF which has 4 columns (iteration number, filename, start-time, end-time). Each iteration a df is made with the three first ones filled in and the last as NA, when it ends then this is filled in with the time and the df merged with the rest of the df. You should save to drive each iteration this df and the data. As well as overwrite there is the `continue` flag, which uses the filename (in CHB it could be the iteration number?) to continue adding the missing data.

In [ ]:
# Run the code and create/overwite old data?
UPENN_OVERWRITE = True

# Make the frame for all patients or just patient 2 (False)?
UPENN_ALL_PART = False

# Where to save the feature data
UPENN_SAVE_PATH = '/content/drive/My Drive/Data/UPennMayo_features.hdf5'

if UPENN_ALL_PART:
  UPENN_SAVE_PATH = '/content/drive/My Drive/Data/UPennMayo_all_features.hdf5'

**Timings**
- The single participant file takes about 30 minutes

**NOTE**
- hickle is an interesting way of storing data
- origionally I used *.to_hdf()* on the pandas object and appended each to the bottom of a table. However this errored when adding in the Hill data because there was too many columns:

    "HDF5 has a header limit of 64kb for all metadata of the columns. This include name, types, etc. When you go about roughly 2000 columns, you will run out of space to store all the metadata."
    
    This doesnt exist if you do it manually like I do below

In [ ]:
# set mne to only output warnings
mne.set_log_level('WARNING')

def save_to_database(save_dir, part_id, combined_df, condition):

  # open the file in append mode (make it if doesnt exist)
  h5file = tables.open_file(save_dir, mode="a", title="Upenn Features")

  # if there is already a node for the particpant...
  if "/"+part_id in h5file:
    # ...put in the directory of where it is found
    part_x_array = h5file.get_node("/" + part_id + '/Data_x')
    part_y_array = h5file.get_node("/" + part_id + '/Data_y')

    # Make sure the columns are in the same order.
    # This is for when there are electrodes which have 'died'
    # which happens sometimes
    data_x_labels = h5file.get_node('/'+part_id+'/Data_x_Feat_Names')
    combined_df = combined_df.reindex(data_x_labels[:].astype(str), axis=1)

  else:
    # create the group directory
    part_group = h5file.create_group("/", part_id, 'Participant Data')
    # make an atom which has the datatype found in the data we want to store
    x_atom = tables.Atom.from_dtype(combined_df.values.dtype)
    y_atom = tables.Atom.from_dtype(condition.dtype)

    # create an array we can append onto later
    part_x_array = h5file.create_earray("/" + part_id,            # parentnode
                                        'Data_x',                 # name 
                                        x_atom,                   # atom
                                        (0,combined_df.shape[1]), # shape
                                        'Feature Array')          # title

    part_y_array = h5file.create_earray("/" + part_id, 
                                        'Data_y', 
                                        y_atom, 
                                        (0,1),
                                        'Events Array')

    # create the feature names array (we only need to do this once)
    h5file.create_array("/" + part_id,                               # where
                        'Data_x_Feat_Names',                         # name 
                        np.array(combined_df.columns, dtype='unicode'),  # obj
                        "Names of Each Feature")                     # title

  # append the data to the array directory
  part_x_array.append(combined_df.values)
  part_y_array.append(condition)

  # flush the data to disk
  h5file.flush()
  # close the file
  h5file.close()

def mat_to_df(file_path, output = False):
  mat = loadmat(file_path)    # load mat-file

  data = mat['data']          # variable in mat file
  channels = mat['channels']  # dtypes of structures are "unsized objects"
  freq = mat['freq'][0]

  channels_list = []
  for channel_array in channels[0][0]:
    channels_list.append(channel_array[0])

  df = pd.DataFrame(data,
                    index=channels_list)

  df = df.T

  # remove columns that do not change value
  df = df.loc[:, (df != df.iloc[0]).any()]

  if output:
    display(df.head())

  return df, freq

def file_class(file_name):

  if re.findall('interictal', file_name):
    return 0
  elif re.findall('ictal', file_name):
    return 1
  else:
    return 2


if UPENN_OVERWRITE:

  if os.path.exists(UPENN_SAVE_PATH):
    os.remove(UPENN_SAVE_PATH)

  # -------------
  # SET UP KAGGLE
  # -------------
  user = getpass('Kaggle Username: ')
  key = getpass('Kaggle API key: ')

  if '.kaggle' not in os.listdir('/root'):
      !mkdir ~/.kaggle
  !touch /root/.kaggle/kaggle.json
  !chmod 666 /root/.kaggle/kaggle.json
  with open('/root/.kaggle/kaggle.json', 'w') as f:
      f.write('{"username":"%s","key":"%s"}' % (user, key))
  !chmod 600 /root/.kaggle/kaggle.json

  # ----------------
  # DOWNLOAD & UNZIP
  # ----------------
  if not os.path.exists('clips.tar.gz'):
    !kaggle competitions download -c seizure-detection
  
  if not os.path.exists("Volumes"):
    if UPENN_ALL_PART:
      !tar xvzf clips.tar.gz
    else:
      !tar xvzf clips.tar.gz Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/
  
  upenn_dir = os.path.join(os.getcwd(), 'Volumes', 'Seagate', 'seizure_detection', 
                           'competition_data', 'clips')
  # ---------------
  # CREATE FEATURES
  # ---------------
  for part_dir in tqdm(file_list(os.path.join(upenn_dir, '*')), desc='Patient Folders'):
    part_file_list = file_list(os.path.join(part_dir, '*'), output=False)
    part_id = re.findall('Patient_\d+', part_file_list[0])[0]

    for file in tqdm(part_file_list, desc='Patient Files'):
      df, freq = mat_to_df(file)
      class_name = file_class(file)
      
      feat = Seizure_Features(sf = freq, 
                              window_size=None,
                              bandpasses = [[2,4],[4,8],[8,12],
                                            [12,30],[30,70]],
                              feature_list=['power', 'power_ratio', 'mean', 'mean_abs', 
                                            'std', 'ratio', 'LSWT', 'fft_corr', 'fft_eigen',
                                            'time_corr', 'time_eigen'])
      part_x_feat = feat.transform(df.values, 
                                   channel_names_list = list(df.columns))
      
      part_x_feat = pd.DataFrame(part_x_feat, columns = feat.feature_names)
      
      part_y_feat = np.expand_dims(np.array([class_name]), axis=1)
        
      save_to_database(UPENN_SAVE_PATH, part_id, 
                       part_x_feat, part_y_feat)

Streaming output truncated to the last 5000 lines.
Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/Patient_2_interictal_segment_2695.mat
Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/Patient_2_interictal_segment_2696.mat
Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/Patient_2_interictal_segment_2697.mat
Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/Patient_2_interictal_segment_2698.mat
Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/Patient_2_interictal_segment_2699.mat
Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/Patient_2_interictal_segment_27.mat
Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/Patient_2_interictal_segment_270.mat
Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/Patient_2_interictal_segment_2700.mat
Volumes/Seagate/seizure_detection/competition_data/clips/Patient_2/Patient_2_interictal_segment_2701.mat
Volumes

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:133: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:137: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


## CHB-MIT Scalp EEG Database

Here are some settings. This just allows control over whether to create a raw or feature dataframe, and whether to extract for all patients or just the first.

**Timing**
- For both the filtered and feature dataframe (for 1 patient) it takes about 2hrs (depends on the machine you get)
- The filtered df is much quicker to create

**NOTE**
- we only use the first patient for the tutorials

In [ ]:
# Run the code and create/overwite old data?
CHB_FILT_OVERWRITE = True
CHB_FEAT_OVERWRITE = True

# Make the frame for all patients or just 1 (False)?
CHB_ALL_PART = False

# Where to save the feature data
CHB_FILT_SAVE_PATH = '/content/drive/My Drive/Data/CHB_filt.hdf5'
CHB_FEAT_SAVE_PATH = '/content/drive/My Drive/Data/CHB_feat.hdf5'

if CHB_ALL_PART:
  CHB_FILT_SAVE_PATH = '/content/drive/My Drive/Data/CHB_all_filt.hdf5'
  CHB_FEAT_SAVE_PATH = '/content/drive/My Drive/Data/CHB_all_feat.hdf5'

In [ ]:
def get_content(part_code):
  url = "https://physionet.org/physiobank/database/chbmit/"+part_code+'/'+part_code+'-summary.txt'
  filename = "./chbmit.txt"

  urlretrieve(url,filename)

  # read the file into a list
  with open(filename, encoding='UTF-8') as f:
      # read all the document into a list of strings (each line a new string)
      content = f.readlines()
      os.remove(filename)

  return content

  import re

def info_dict(content):
  
  line_nos=len(content)
  line_no=1

  channels = []
  file_name = []
  file_info_dict={}

  for line in content:

    # if there is Channel in the line...
    if re.findall('Channel \d+', line):
      # split the line into channel number and channel reference
      channel = line.split(': ')
      # get the channel reference and remove any new lines
      channel = channel[-1].replace("\n", "")
      # put into the channel list
      channels.append(channel)

    # if the line is the file name
    elif re.findall('File Name', line):
      # if there is already a file_name
      if file_name:
        # flush the current file info to it
        part_info_dict[file_name] = file_info_dict

      # get the file name
      file_name = re.findall('\w+\d+_\d+|\w+\d+\w+_\d+', line)[0]

      file_info_dict = {}
      # put the channel list in the file info dict and remove duplicates
      file_info_dict['Channels'] = list(set(channels))
      # reset the rest of the options
      file_info_dict['Start Time'] = ''
      file_info_dict['End Time'] = ''
      file_info_dict['Seizures Window'] = []

    # if the line is about the file start time
    elif re.findall('File Start Time', line):
      # get the start time
      file_info_dict['Start Time'] = re.findall('\d+:\d+:\d+', line)[0]

    # if the line is about the file end time
    elif re.findall('File End Time', line):
      # get the start time
      file_info_dict['End Time'] = re.findall('\d+:\d+:\d+', line)[0]

    elif re.findall('Seizure Start Time|Seizure End Time|Seizure \d+ Start Time|Seizure \d+ End Time', line):
      file_info_dict['Seizures Window'].append(int(re.findall('\d+', line)[-1]))

    # if last line in the list...
    if line_no == line_nos:
      # flush the file info to it
      part_info_dict[file_name] = file_info_dict

    line_no+=1
    

def data_load(file, selected_channels=[]):

  try: 
    url = "https://physionet.org/physiobank/database/chbmit/"+file
    filename = "./chbmit.edf"

    urlretrieve(url,filename)
    # use the reader to get an EdfReader file
    f = pyedflib.EdfReader(filename)
    os.remove(filename)
    
    # get a list of the EEG channels
    if len(selected_channels) == 0:
      selected_channels = f.getSignalLabels()

    # get the names of the signals
    channel_names = f.getSignalLabels()
    # get the sampling frequencies of each signal
    channel_freq = f.getSampleFrequencies()

    # make an empty file of 0's
    sigbufs = np.zeros((f.getNSamples()[0],len(selected_channels)))
    # for each of the channels in the selected channels
    for i, channel in enumerate(selected_channels):
      # add the channel data into the array
      sigbufs[:, i] = f.readSignal(channel_names.index(channel))
    
    # turn to a pandas df and save a little space
    df = pd.DataFrame(sigbufs, columns = selected_channels).astype('float32')
    
    # get equally increasing numbers upto the length of the data depending
    # on the length of the data divided by the sampling frequency
    index_increase = np.linspace(0,
                                  len(df)/channel_freq[0],
                                  len(df), endpoint=False)

    # round these to the lowest nearest decimal to get the seconds
    seconds = np.floor(index_increase).astype('uint16')

    # make a column the timestamp
    df['Time'] = seconds

    # make the time stamp the index
    df = df.set_index('Time')

    # name the columns as channel
    df.columns.name = 'Channel'

    return df, channel_freq[0]

  except:
    OSError
    return pd.DataFrame(), None


def create_events(file_name, df):

  # default df with all 0's the length of the last time datapoint
  events_df = pd.DataFrame([0]*int(df.iloc[-1].name+1))

  if file_name in part_info_dict:
    seizures_window = part_info_dict[file_name]['Seizures Window']
  else:
    seizures_window = []

  # if not empty
  if seizures_window:
    # just get the start values
    seizures_start = seizures_window[0::2]
    # just get the end values
    seizures_end = seizures_window[1::2]
    # get index values where seizures occour
    seizures_index = []
    for index, value in enumerate(seizures_start):
      seizures_index.extend(list(range(value,seizures_end[index])))

    # make a df of 1's with the index where there are seizures
    seizure_events_df = pd.DataFrame([1]*len(seizures_index),seizures_index)

    # update values where seizure
    events_df.update(seizure_events_df, overwrite = True)

  # name the index 'time'
  events_df.index.name='Time'
  # name the column event
  events_df.columns=['Class']

  events_df = events_df.astype('uint8')
  
  # join the events to match the time they are associated with
  df = df.join(events_df, on='Time')

  # get the events
  data_y = df['Class']

  return data_y

def window_y(events, window_size, overlap):
    
  # window the data so each row is another epoch
  events_windowed = window(events, w = window_size, o = overlap, copy = True)

  # get the value most frequent in the window
  #data_y_mode = stats.mode(events_windowed, axis=1)[0].flatten()
  
  # turn to array of bools if seizure in the
  # windowed data
  bools = events_windowed == 1
  # are there any seizure seconds in the data?
  data_y = np.any(bools,axis=1)
  # turn to 0's and 1's
  data_y = data_y.astype(int)

  
  return data_y

def save_to_database(save_dir, file_title, group, data_x, data_y, feature_columns):

    # open the file in append mode (make it if doesnt exist)
    h5file = tables.open_file(save_dir, mode="a", title=file_title)
    
    # save space
    data_x = data_x.astype(np.float32)
    data_y = data_y.astype(np.int16)
    
    # get filters to compress file
    filters = tables.Filters(complevel=1, complib='zlib')
    
    # if there is already a node for the particpant...
    if "/"+group in h5file:
        # ...put in the directory of where it is found
        part_x_array = h5file.get_node("/" + group + '/Data_x')
        part_y_array = h5file.get_node("/" + group + '/Data_y')
    
    else:
        # create the group directory
        part_group = h5file.create_group("/", group, 'Group Data')
        # make an atom which has the datatype found in the data we want to store
        x_atom = tables.Atom.from_dtype(data_x.dtype)
        y_atom = tables.Atom.from_dtype(data_y.dtype)

        # create an array we can append onto later
        part_x_array = h5file.create_earray("/" + group,                   # parentnode
                                            'Data_x',                        # name 
                                            x_atom,                          # atom
                                            (0,data_x.shape[1], data_x.shape[2]), # shape
                                            'Feature Array',
                                            filters=filters
                                           )                 # title

        part_y_array = h5file.create_earray("/" + group, 
                                            'Data_y', 
                                            y_atom, 
                                            (0,),
                                            'Events Array',
                                            filters=filters
                                           )
        
        # create the feature names array (we only need to do this once)
        h5file.create_array("/" + group,                                   # where
                            'Feature_Names',                             # name 
                            np.array(feature_columns, dtype='unicode'),  # obj
                            "Names of Each Feature")                         # title
    
    # append the data to the array directory
    part_x_array.append(data_x)
    part_y_array.append(data_y)
    
    # flush the data to disk
    h5file.flush()
    # close the file
    h5file.close()


if CHB_FILT_OVERWRITE or CHB_FEAT_OVERWRITE:

  if CHB_FILT_OVERWRITE and os.path.exists(CHB_FILT_SAVE_PATH):
    os.remove(CHB_FILT_SAVE_PATH)

  if CHB_FEAT_OVERWRITE and os.path.exists(CHB_FEAT_SAVE_PATH):
    os.remove(CHB_FEAT_SAVE_PATH)

  dbs = wfdb.get_dbs()

  records_list = wfdb.io.get_record_list('chbmit', records='all')
  part_codes = sorted(list(set([record.split('/')[0] for record in records_list])))

  # these are channels common to all records
  channel_keeps = ['P7-O1','FP2-F4','F7-T7','FP1-F3','F4-C4','FP2-F8','P8-O2',
                  'C4-P4','F3-C3','T8-P8','P3-O1','FZ-CZ','FP1-F7','P4-O2',
                  'T7-P7','CZ-PZ','F8-T8','C3-P3']
  
  # ----------------------------
  # PARTICIPANT INFORMATION DICT
  # ----------------------------
  part_info_dict = {}
  for part_code in part_codes:
    content = get_content(part_code)
    info_dict(content)

  # fix wrong times
  start_time_24 = part_info_dict['chb03_24']['Start Time']
  end_time_24 = part_info_dict['chb03_24']['End Time']
  start_time_25 = part_info_dict['chb03_25']['Start Time']
  end_time_25 = part_info_dict['chb03_25']['End Time']
  part_info_dict['chb03_24']['Start Time'] = start_time_25
  part_info_dict['chb03_24']['End Time'] = end_time_25
  part_info_dict['chb03_25']['Start Time'] = start_time_24
  part_info_dict['chb03_25']['End Time'] = end_time_24

  # ---------------
  # CREATE FEATURES
  # ---------------
  # reduce the list to just the first participant
  if not CHB_ALL_PART:
    regex = re.compile('chb01')
    records_list = [i for i in records_list if regex.search(i)]

  for record in tqdm(records_list):
    part_id = record.split('/')[0]
    file_name = record.split('/')[1].split('.')[0]
    raw_data, freq = data_load(record, channel_keeps)
    if raw_data.empty:
      print('Skipped: '+file_name)
    else:
      raw_events = create_events(file_name, raw_data)
      
      if CHB_FILT_OVERWRITE:
        # filter the data
        b, a = signal.butter(4, [1/(freq/2), 30/(freq/2)], 'bandpass', analog=False)
        filt_x = signal.filtfilt(b, a, raw_data.T).T
        
        # scale the data
        SS = StandardScaler()
        scaled_data = SS.fit_transform(filt_x)
        scaled_data = pd.DataFrame(scaled_data, columns = raw_data.columns, 
                                   index = raw_events)
        # drop na
        scaled_data = scaled_data.dropna()
        
        # window the data
        window_size = 256*2
        overlap = 256
        data_x = window_x(scaled_data, window_size, overlap)
        data_y = window_y(raw_events, window_size, overlap)
        
        # save the data
        save_to_database(CHB_FILT_SAVE_PATH,
                        'CHB_filt',
                        part_id,
                        data_x,
                        data_y, 
                        list(scaled_data.columns))
      
      if CHB_FEAT_OVERWRITE:
        feat = Seizure_Features(sf = freq,
                                window_size=2,
                                overlap=1,
                                levels=6,
                                bandpasses = [[1,4],[4,8],[8,12],
                                              [12,30],[30,70]],
                                feature_list=['power', 'power_ratio', 
                                              'mean', 
                                              'mean_abs', 'std', 'ratio', 
                                              'LSWT', 
                                              'fft_corr', 'fft_eigen', 'time_corr', 
                                              'time_eigen'],
                                scale = True)
        with warnings.catch_warnings():
          warnings.filterwarnings("ignore", category=RuntimeWarning)
          x_feat, y_feat = feat.transform(raw_data.values, 
                                          raw_events.values,
                                          channel_names_list = list(raw_data.columns))

        
        # TODO: sometimes the feats come out as Float64Index...
        # should probably look into why that is, but for now, I'll just
        # turn them to ints
        feat_df = pd.DataFrame(x_feat,
                               index=y_feat[:,0].astype(int), 
                               columns = feat.feature_names)

        feat_df = feat_df.dropna()
        
        feat_df.to_hdf(CHB_FEAT_SAVE_PATH, part_id, format='table', append=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:343: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:254: FutureWarning: Series.strides is deprecated and will be removed in a future version
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:254: FutureWarning: Series.strides is deprecated and will be removed in a future version
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:254: FutureWarning: Series.strides is deprecated and will be removed in a future version
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:254: FutureWarning: Series.strides is deprecated and will be removed in a future version
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:254: FutureWarning: Series.strides is deprecated and will be removed in a future version
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:254: FutureWarning: Series.strides is deprecated and will be removed in a future version
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:254: FutureWarning: Series.strides is d

## NEDC TUH EEG Seizure corpus

**Timings**
- For the UDWT, filtered, and feature dataframes (for absence seizures) it takes about 20 mins

**TODO**
- change the way data is downloaded to be alike to the CHB data (download it, load it, remove it), rather than currently where it downloads all the files. This is fine for the smaller sets but for largers ones (such as all seizure types) it will fill the disk and crash.

In [28]:
# Run the code and create/overwite old data?
TUH_FILT_OVERWRITE = True
TUH_FEAT_OVERWRITE = True
TUH_UDWT_OVERWRITE = True

# What type of seizure in TUH (If `None` will do all)?
TUH_code = 'ABSZ'

# Where to save the feature data
TUH_FILT_SAVE_PATH = '/content/drive/My Drive/Data/TUH_filt.hdf5'
TUH_FEAT_SAVE_PATH = '/content/drive/My Drive/Data/TUH_feat.hdf5'
TUH_UDWT_SAVE_PATH = '/content/drive/My Drive/Data/TUH_udwt.hdf5'

if not TUH_code:
  TUH_FILT_SAVE_PATH = '/content/drive/My Drive/Data/TUH_all_filt.hdf5'
  TUH_FEAT_SAVE_PATH = '/content/drive/My Drive/Data/TUH_all_feat.hdf5'
  TUH_UDWT_SAVE_PATH = '/content/drive/My Drive/Data/TUH_all_udwt.hdf5'

**NOTES**
- I don't re-reference, instead leaving the origional references in. This means that I select files with the most common type of reference for that seizure.
- I merge the files below to the same participants together. They are still windowed separately but it just means when you split the training and test its easier to ensure data from the same patient doesnt fall into separate test and training.
- Getting a warning for getting features on records like 00003281:
`RuntimeWarning: All-NaN slice encountered`
but this is just at for a couple of seconds at the start of the records. Probably to do with electrode noise so I'm just going to ignore it for now.

In [30]:
def find_files(url, headers):
    # get a soup of the directory url
    soup = BeautifulSoup(requests.get(url, auth=(headers['user'], headers['passwd'])).text, 
                         features="html.parser")
    # make a list of all the links in the url
    hrefs_list = []
    for link in soup.find_all('a'):
        hrefs_list.append(link.get('href'))

    return hrefs_list
    
    
def download_file(download_file_url, file_path, headers, output=False):
    if output:
        # print it is downloading
        print('Downloading: '+ download_file_url)
    # download the file to the directory
    r = requests.get(download_file_url, auth=(headers['user'], headers['passwd']))
    with open(file_path, 'wb') as f:
      f.write(r.content)

# needs a directory to download it to
def download_TUH(DIR, headers, sub_dir, output=False):
    
    # directory url
    dir_url = 'https://www.isip.piconepress.com/projects/tuh_eeg/downloads/tuh_eeg_seizure/v1.5.0/'+sub_dir

    hrefs_dir_list = find_files(dir_url, headers)
    
    # for each link in the directory
    for link in hrefs_dir_list:
        # download the files outside of participant folders we want
        if re.findall('.xlsx|\.edf|\.tse(?!_)', str(link)):
            # if the file doesnt already exist in the directory
            if not os.path.exists(os.path.join(DIR, link)):
                download_file(dir_url+'/'+str(link), DIR+'/'+str(link), headers, output)

def data_load(data_file, selected_channels=[]):

    try:
        # use the reader to get an EdfReader file
        f = pyedflib.EdfReader(data_file)

        # get the names of the signals
        channel_names = f.getSignalLabels()
        # get the sampling frequencies of each signal
        channel_freq = f.getSampleFrequencies()
        
        # get a list of the EEG channels
        if len(selected_channels) == 0:
            selected_channels = channel_names

        # make an empty file of 0's
        sigbufs = np.zeros((f.getNSamples()[0],len(selected_channels)))
        # for each of the channels in the selected channels
        for i, channel in enumerate(selected_channels):
        
            try:
              # add the channel data into the array
              sigbufs[:, i] = f.readSignal(channel_names.index(channel))
            
            except:
              ValueError
              # This happens if the sampling rate of that channel is 
              # different to the others.
              # For simplicity, in this case we just make it na.
              sigbufs[:, i] = np.nan

        # turn to a pandas df and save a little space
        df = pd.DataFrame(sigbufs, columns = selected_channels).astype('float32')

        # get equally increasing numbers upto the length of the data depending
        # on the length of the data divided by the sampling frequency
        index_increase = np.linspace(0,
                                      len(df)/channel_freq[0],
                                      len(df), endpoint=False)

        # round these to the lowest nearest decimal to get the seconds
        seconds = np.floor(index_increase).astype('uint16')

        seconds = index_increase
        
        # make a column the timestamp
        df['Time'] = seconds

        # make the time stamp the index
        df = df.set_index('Time')

        # name the columns as channel
        df.columns.name = 'Channel'

        return df, channel_freq[0]

    except OSError as error:
        print('Error '+data_file+': '+str(error))
        return pd.DataFrame(), None
    except ValueError as error:
        print('Error '+data_file+': '+str(error))
        return pd.DataFrame(), None

def create_events(file_name, df, code = None):

    data_y = pd.Series(index=df.index)
    data_y.name = 'Events'

    events_tse = pd.read_csv(file_name, 
                             skiprows=1,
                             sep = ' ',
                             header=None,
                             names =['Start', 'End', 'Code', 'Certainty'])
    
    data_y = data_y.fillna('bckg')
    
    for pos, row in events_tse.iterrows():
        # if you want to manually set the code
        if code != None:
          if row['Code'] == code:
              data_y[row['Start']:row['End']] = code
        # let it be the code it is in the event file
        else:
          data_y[row['Start']:row['End']] = row['Code']

    return data_y

def window_y(events, window_size, overlap, target=None, baseline=None):
    
  # window the data so each row is another epoch
  events_windowed = window(events, w = window_size, o = overlap, copy = True)
  
  if target:
    # turn to array of bools if seizure in the
    # windowed data
    bools = events_windowed == target
    # are there any seizure seconds in the data?
    data_y = np.any(bools,axis=1)
    # turn to 0's and 1's
    data_y = data_y.astype(int)
    # expand the dimensions so running down one column
    data_y = np.expand_dims(data_y, axis=1)
  
  elif baseline:
    # replace all baseline labels to nan
    data_y = pd.DataFrame(events_windowed).replace(baseline, np.nan)
    # get the most common other than baseline
    data_y = data_y.mode(1)
    # change nan back to baseline class
    data_y = data_y.fillna(baseline).values
    # if there was nothing but baseline there will be an empty array
    if data_y.size == 0:
        data_y = np.array([baseline]*data_y.shape[0])
        data_y = np.expand_dims(data_y, -1)
  
  else:
    # get the value most frequent in the window
    data_y = pd.DataFrame(events_windowed).mode(1).values

  return data_y

def downsample(data_x, data_y, freq):
    if freq > 256:
        if freq >= 1000:
          subsample = 4
        else:
          subsample = 2

        freq = freq/subsample
        data_x = data_x[::subsample]
        data_y = data_y[::subsample]

    return data_x, data_y, freq

def sel_file_list(set_name, seiz_type):
  # load the training information
  train_info = pd.read_excel('/content/TUH Database/seizures_v32r.xlsx', set_name)
  # just want the info per file here
  file_info = train_info.iloc[1:6101,1:15]
  # cleans some of the names
  file_info_cols = ['File No.', 'Patient', 'Session', 'File', 
                        'EEG Type', 'EEG SubType', 'LTM or Routine', 
                        'Normal/Abnormal', 'No. Seizures File', 
                        'No. Seizures/Session', 'Filename', 'Seizure Start', 
                        'Seizure Stop', 'Seizure Type']
  file_info.columns = file_info_cols

  # we forward fill as there are gaps in the excel file to represent the info 
  # is the same as above (apart from in the filename, seizure start, seizure stop 
  # and seizure type columns)
  for col_name in file_info.columns[:-4]:
    file_info[col_name] = file_info[col_name].ffill()

  # patient ID is an integer rather than float
  file_info['Patient'] = file_info['Patient'].astype(int)

  if seiz_type:
    # Get a list of patient event files that have a specifc type of seizure
    return list(file_info[file_info['Seizure Type']==seiz_type]['Filename'])
  else:
    return list(file_info['Filename'])

def save_to_database(save_dir, file_title, group, data_x, data_y, 
                     feature_columns=None):

    # open the file in append mode (make it if doesnt exist)
    h5file = tables.open_file(save_dir, mode="a", title=file_title)
    
    # save space
    data_x = data_x.astype(np.float32)
    #data_y = data_y.astype(np.int16)
    
    # get filters to compress file
    filters = tables.Filters(complevel=1, complib='zlib')
    
    # if there is already a node for the particpant...
    if "/"+group in h5file:
        # ...put in the directory of where it is found
        part_x_array = h5file.get_node("/" + group + '/Data_x')
        part_y_array = h5file.get_node("/" + group + '/Data_y')
    
    else:
        # create the group directory
        part_group = h5file.create_group("/", group, 'Group Data')
        # make an atom which has the datatype found in the data we want to store
        x_atom = tables.Atom.from_dtype(data_x.dtype)
        y_atom = tables.Atom.from_dtype(data_y.dtype)

        if file_title == 'UDWT_Data':
          shape = (0,data_x.shape[1], data_x.shape[2], data_x.shape[3])
        else:
          shape = (0,data_x.shape[1], data_x.shape[2])
        
        # create an array we can append onto later
        part_x_array = h5file.create_earray("/" + group,                   # parentnode
                                            'Data_x',                        # name 
                                            x_atom,                          # atom
                                            shape, # shape
                                            'Feature Array',
                                            filters=filters
                                           )                 # title

        part_y_array = h5file.create_earray("/" + group, 
                                            'Data_y', 
                                            y_atom, 
                                            (0,),
                                            'Events Array',
                                            filters=filters
                                           )
        
        if feature_columns:
          # create the feature names array (we only need to do this once)
          h5file.create_array("/" + group,                                   # where
                              'Feature_Names',                             # name 
                              np.array(feature_columns, dtype='unicode'),  # obj
                              "Names of Each Feature")                         # title
    
    # append the data to the array directory
    part_x_array.append(data_x)
    part_y_array.append(data_y)
    
    # flush the data to disk
    h5file.flush()
    # close the file
    h5file.close()

def udwt_spectrogram(data, waveletname, level, window_size):
  data_ucwt = np.ndarray(shape=(data.shape[0], level, data.shape[1], data.shape[2]))
  
  for ii in range(data.shape[0]):
    for jj in range(data.shape[-1]):
      signal = data[ii, :, jj]
      coeffs_list = swt(signal, waveletname, level=level)
      
      coeffs_array = np.zeros((len(coeffs_list), window_size))
      
      for i, array_tuple in enumerate(coeffs_list[::-1]):

        coeffs_array[i,:] = np.array(array_tuple)[1,:]
      
      power = np.abs(coeffs_array)**2
      
      data_ucwt[ii, :, :, jj] = power
      
  return data_ucwt


if TUH_FILT_OVERWRITE or TUH_FEAT_OVERWRITE or TUH_UDWT_OVERWRITE:

  if TUH_FILT_OVERWRITE and os.path.exists(TUH_FILT_SAVE_PATH):
    os.remove(TUH_FILT_SAVE_PATH)

  if TUH_FEAT_OVERWRITE and os.path.exists(TUH_FEAT_SAVE_PATH):
    os.remove(TUH_FEAT_SAVE_PATH)

  if TUH_UDWT_OVERWRITE and os.path.exists(TUH_UDWT_SAVE_PATH):
    os.remove(TUH_UDWT_SAVE_PATH)

  # ---------
  # TUH SETUP
  # ---------
  DOWNLOAD_DIR = "TUH Database"
  if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)

  user = getpass('TUH Username: ')
  key = getpass('TUH Password: ')
  auth_dict = {'user': user, 'passwd': key}

  # --------------
  # GET FILE PATHS
  # --------------
  # download info files
  download_TUH(DOWNLOAD_DIR, auth_dict, '_DOCS')

  seiz_types_path = '/content/TUH Database/seizures_types_v01.xlsx'
  seiz_types = pd.read_excel(seiz_types_path)

  seiz_types = seiz_types.set_index('Class Code')

  int_code = seiz_types.to_dict()['Class No.']
  # change to lower case
  int_code = { k.lower() : v for k,v in int_code.items() if not isinstance(k, float)}

  # get a list of files
  tuh_file_list = sel_file_list('train', TUH_code)+sel_file_list('dev_test', TUH_code)

  # get a list of the montages
  montage = []
  for file in tuh_file_list:
    montage.append(file.split('/')[3])

  # count how many times the montages appear in the data
  montage_counts = pd.Series(montage).value_counts()

  # remove all files apart from those in the most common montage
  regex = re.compile(montage_counts.index[0])
  tuh_file_list = [i for i in tuh_file_list if regex.search(i)]
  # remove duplicates
  tuh_file_list = list(set(tuh_file_list))

  # They changed "dev_test" to just "dev" in their file structure so 
  # I need to account for this now...
  for i, string in enumerate(tuh_file_list):
    tuh_file_list[i] = re.sub('_test', '', string)


  # --------------------
  # GET SIMILAR CHANNELS
  # --------------------
  # this is to make sure all the data have the same channels
  all_channels = []
  for events_path in tqdm(tuh_file_list, desc = 'Finding Channels'):
    file_ID = events_path.split('/')[-1][:-4]
    # we use the above to get the file directory this file is in
    pat_file_dir = '/'.join(events_path.split('/')[1:-1])
    # this will download all edf and event files for the selected patient
    download_TUH(DOWNLOAD_DIR, auth_dict, pat_file_dir, output=True)
    
    with pyedflib.EdfReader(DOWNLOAD_DIR+'/'+file_ID+'.edf') as f:
        # get the names of the signals
        all_channels.extend(f.getSignalLabels())

  # turn the list into a pandas series
  all_channels = pd.Series(all_channels)

  # count how many times the channels appear in each participant
  channel_counts = all_channels.value_counts()
  
  # threshold the channels to only those found in all raw data
  channel_keeps = list(channel_counts[channel_counts >= channel_counts[0]].index)
  regex = re.compile('30|PHOTIC|EKG|PG')
  channel_keeps = [i for i in channel_keeps if not regex.search(i)]

  # ---------------
  # CREATE FEATURES
  # ---------------
  for events_path in tqdm(tuh_file_list, desc='Creating Features'):
    file_ID = events_path.split('/')[-1][:-4]
    # we use the above to get the file directory this file is in
    pat_file_dir = '/'.join(events_path.split('/')[1:-1])
    # patient ID
    pat_ID = events_path.split('/')[-3]

    # this will download all edf and event files for the selected patient
    #download_TUH(DOWNLOAD_DIR, auth_dict, pat_file_dir, output=True)

    # load data
    raw_data, freq = data_load(DOWNLOAD_DIR+'/'+file_ID+'.edf', channel_keeps)

    if raw_data.empty:
      print('Skipped: '+file_ID)
    else:
      raw_events = create_events(DOWNLOAD_DIR+'/'+file_ID+'.tse', raw_data)
      # change to integer representation
      raw_events = raw_events.replace(int_code)

      if TUH_FILT_OVERWRITE or TUH_UDWT_OVERWRITE:

        # downsample
        down_x, down_y, down_freq = downsample(raw_data.values, raw_events.values, freq)

        # window y
        window_size = 256*2
        overlap = 256
        
        if TUH_FILT_OVERWRITE:
          # filter the data
          b, a = signal.butter(4, [1/(down_freq/2), 30/(down_freq/2)], 'bandpass', analog=False)
          filt_data = signal.filtfilt(b, a, down_x.T).T
          
          # scale the data over each channel
          SS = StandardScaler()
          scaled_data = SS.fit_transform(filt_data)
          scaled_data = pd.DataFrame(scaled_data, columns = raw_data.columns, index = down_y)
          
          # drop na
          scaled_data = scaled_data.dropna()
          
          # window x
          data_x = window_x(scaled_data, window_size, overlap)
          data_y = window_y(scaled_data.index.values, window_size, overlap, target=None, 
                            baseline=6)
          
          # to stop it printing warnings like: object name is not a valid 
          # Python identifier: '00001113'
          with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=tables.NaturalNameWarning)
            # save the data
            save_to_database(TUH_FILT_SAVE_PATH, 'Filt_Data', pat_ID, data_x, 
                            data_y[:,0], list(scaled_data.columns))
          
        # TODO: add in a dropna bit
        if TUH_UDWT_OVERWRITE:
          # window x
          data_x = window_x(pd.DataFrame(down_x, columns = raw_data.columns), 
                            window_size, overlap)
          data_y = window_y(down_y, window_size, overlap, target=None, 
                          baseline=6)

          udwt_data = udwt_spectrogram(data_x, 'db4', 6, window_size)

          # get the shape of this data
          orig_shape = udwt_data.shape
          # reshape to merge the levels and data
          udwt_data_reshape = np.reshape(udwt_data, (-1, orig_shape[-1]))
          # scale across channels
          SS = StandardScaler()
          udwt_data_scaled = SS.fit_transform(udwt_data_reshape)
          # shape the data back
          udwt_data_scaled = np.reshape(udwt_data_scaled, orig_shape)

          #display(udwt_data_scaled.shape)

          # to stop it printing warnings like: object name is not a valid 
          # Python identifier: '00001113'
          with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=tables.NaturalNameWarning)
            # save the data
            save_to_database(TUH_UDWT_SAVE_PATH, 'UDWT_Data', 
                            pat_ID,
                            #file_ID, # had to use file ID as appending to an existing array was not working 
                            udwt_data_scaled, data_y[:,0])

        
      if TUH_FEAT_OVERWRITE:
        # TODO: in 00001795 relative_power = bandpass_2/bandpass_1 creates an inf
        # due to dividing by 0. Fix this with a checks in the function, but for now,
        # for a quick patch I'll just not use power_ratio.
        feat = Seizure_Features(sf = freq,
                                window_size=2,
                                overlap=1,
                                levels=6,
                                bandpasses = [[1,4],[4,8],[8,12],
                                              [12,30],[30,70]],
                                feature_list=['power', #'power_ratio', 
                                              'mean', 'mean_abs', 
                                              'std', 'ratio', 'LSWT', 'fft_corr', 
                                              'fft_eigen', 'time_corr', 'time_eigen'],
                                scale = True,
                                baseline=6)
        

        
        # just to ignore the runtime warnings about na's
        with warnings.catch_warnings():
          warnings.filterwarnings("ignore", category=RuntimeWarning)
          
          x_feat, y_feat = feat.transform(raw_data.values, 
                                          raw_events.values,
                                          channel_names_list = list(raw_data.columns))

        
        feat_df = pd.DataFrame(x_feat,
                                index=y_feat[:,0], 
                                columns = feat.feature_names)

        feat_df = feat_df.dropna()
        
        
        # to stop it printing warnings like: object name is not a valid 
        # Python identifier: '00001113'
        with warnings.catch_warnings():
          warnings.filterwarnings("ignore", category=tables.NaturalNameWarning)
          feat_df.to_hdf(TUH_FEAT_SAVE_PATH, 
                        pat_ID,
                        format='table', append=True)

TUH Username: ··········
TUH Password: ··········


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:350: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:375: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.strides is deprecated and will be removed in a future version
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.strides is deprecated and will be removed in a future version
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.strides is deprecated and will be removed in a future version
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.strides is deprecated and will be removed in a future version
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.strides is deprecated and will be removed in a future version
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.strides is deprecated and wil